# Jugend musiziert Data Analysis

Analysis of the scraped Jugend musiziert tournament data using Polars for efficient data processing and exploration.

In [1]:
# Import Required Libraries
import polars as pl
import json
import os
from pathlib import Path

# Display settings
pl.Config.set_fmt_str_lengths(100)
print(f"Polars version: {pl.__version__}")

Polars version: 1.37.1


## Load Scraped Data

Loading the JSON data from the scraper output.

In [2]:
# Load the scraped JSON data
data_path = Path("jugend_musiziert_data.json")

if data_path.exists():
    with open(data_path, 'r', encoding='utf-8') as f:
        raw_data = json.load(f)
    print(f"✓ Loaded data from {data_path}")
    print(f"  Top-level keys: {list(raw_data.keys())}")
else:
    print(f"✗ Data file not found at {data_path}")
    print(f"  Please run the scraper first: uv run python scraper/scraper.py")

✓ Loaded data from jugend_musiziert_data.json
  Top-level keys: ['metadata', 'seasons', 'timetable', 'results']


## Inspect Data Structure

Examine the structure and content of the loaded data.

In [3]:
# Inspect the raw data structure
if 'raw_data' in dir():
    # Display data structure
    print("=" * 60)
    print("DATA STRUCTURE")
    print("=" * 60)
    
    # Check for API endpoints
    if 'api_endpoints' in raw_data:
        endpoints_df = pl.DataFrame({
            'url': raw_data['api_endpoints']
        })
        print(f"\n📍 API Endpoints ({len(endpoints_df)} found):")
        print(endpoints_df)
    
    # Check for embedded data
    if 'embedded_data' in raw_data:
        print(f"\n📊 Embedded Data Blocks: {len(raw_data['embedded_data'])}")
        for i, block in enumerate(raw_data['embedded_data'][:2]):  # Show first 2
            print(f"\n  Block {i+1}:")
            print(f"    Keys: {list(block.keys())}")
            for key, value in block.items():
                if isinstance(value, dict):
                    print(f"    {key}: {list(value.keys())[:5]}...")
                elif isinstance(value, list):
                    print(f"    {key}: list of {len(value)} items")
                else:
                    print(f"    {key}: {type(value).__name__}")
    
    # Check for metadata
    if 'metadata' in raw_data:
        print(f"\n✓ Metadata: {raw_data['metadata']}")

DATA STRUCTURE

✓ Metadata: {'timestamp': '2026-01-25 18:20:24.355814', 'url': 'https://www.jugend-musiziert.org/wettbewerbe/regionalwettbewerbe/baden-wuerttemberg/esslingen-goeppingen-und-rems-murr/zeitplan', 'region': 'esslingen-goeppingen-und-rems-murr', 'api_endpoints': {'seasons': 'https://api.jugend-musiziert.org/api/seasons', 'timetable': 'https://api.jugend-musiziert.org/api/timetable'}}


## Convert to Polars DataFrames

Convert the JSON data into Polars DataFrames for efficient analysis.

In [4]:
# Create DataFrames from API endpoints
if 'raw_data' in dir() and 'api_endpoints' in raw_data:
    endpoints_df = pl.DataFrame({
        'url': raw_data['api_endpoints']
    })
    print("API Endpoints DataFrame:")
    print(f"Shape: {endpoints_df.shape}")
    print(endpoints_df)
    
# Prepare embedded data for analysis
if 'raw_data' in dir() and 'embedded_data' in raw_data:
    print("\n" + "="*60)
    print("EMBEDDED DATA ANALYSIS")
    print("="*60)
    
    # Convert embedded data blocks to Polars
    data_blocks = []
    for i, block in enumerate(raw_data['embedded_data']):
        flat_block = {
            'block_id': i,
            'keys': str(list(block.keys())),
            'has_data': 'data' in block,
        }
        data_blocks.append(flat_block)
    
    if data_blocks:
        blocks_df = pl.DataFrame(data_blocks)
        print(f"\nTotal embedded blocks: {len(blocks_df)}")
        print(blocks_df)

## Exploratory Data Analysis

Analyze the structure and content of the tournament data.

In [5]:
# Analyze the structure of embedded data in detail
if 'raw_data' in dir() and 'embedded_data' in raw_data:
    print("="*60)
    print("DETAILED EMBEDDED DATA ANALYSIS")
    print("="*60)
    
    # Analyze data content
    for i, block in enumerate(raw_data['embedded_data']):
        if 'data' in block and isinstance(block['data'], dict):
            data_content = block['data']
            
            # Count different types of content
            list_keys = [k for k, v in data_content.items() if isinstance(v, list)]
            dict_keys = [k for k, v in data_content.items() if isinstance(v, dict)]
            
            print(f"\nBlock {i+1}:")
            print(f"  Total keys: {len(data_content)}")
            print(f"  List entries: {len(list_keys)} ({', '.join(list_keys[:3])}...)")
            print(f"  Dict entries: {len(dict_keys)} ({', '.join(dict_keys[:3])}...)")
            
            # Analyze list content (potential participants)
            for key in list_keys[:1]:  # Analyze first list
                items = data_content[key]
                if items and isinstance(items, list):
                    print(f"\n  Analyzing '{key}' ({len(items)} items):")
                    if isinstance(items[0], dict):
                        sample_keys = list(items[0].keys())
                        print(f"    Item structure: {sample_keys}")
                    break

## Summary Statistics

Generate summary statistics and insights about the scraped data.

In [6]:
# Generate summary report
if 'raw_data' in dir():
    print("\n" + "="*60)
    print("SUMMARY REPORT")
    print("="*60)
    
    summary = {
        'Metric': [],
        'Value': []
    }
    
    # Count API endpoints
    if 'api_endpoints' in raw_data:
        summary['Metric'].append('API Endpoints Found')
        summary['Value'].append(str(len(raw_data['api_endpoints'])))
    
    # Count embedded data blocks
    if 'embedded_data' in raw_data:
        summary['Metric'].append('Embedded Data Blocks')
        summary['Value'].append(str(len(raw_data['embedded_data'])))
        
        # Count total items across all blocks
        total_items = 0
        for block in raw_data['embedded_data']:
            if 'data' in block and isinstance(block['data'], dict):
                for v in block['data'].values():
                    if isinstance(v, list):
                        total_items += len(v)
        summary['Metric'].append('Total Items Found')
        summary['Value'].append(str(total_items))
    
    # Create summary DataFrame
    if summary['Metric']:
        summary_df = pl.DataFrame(summary)
        print("\n" + summary_df.to_string())
    
    print("\n✓ Data loading and initial analysis complete!")
    print("  Next: Run the cells above to explore the data further.")


SUMMARY REPORT

✓ Data loading and initial analysis complete!
  Next: Run the cells above to explore the data further.


In [7]:
# Extract and display seasons from the raw data
if 'raw_data' in dir() and 'seasons' in raw_data:
    seasons_data = raw_data['seasons']
    
    # Extract the season members
    if 'hydra:member' in seasons_data:
        members = seasons_data['hydra:member']
        
        # Convert to Polars DataFrame
        seasons_df = pl.DataFrame(members)
        
        print("="*60)
        print("SEASONS")
        print("="*60)
        print(f"\nTotal seasons: {seasons_data.get('hydra:totalItems', len(members))}")

        seasons_display = seasons_df.select([
            'id',
            'year',
            'number',
            'defaultRegistrationStart',
            'defaultRegistrationEnd',
            'ageGroups'
        ])

        print("="*60)
        print("SEASONS - KEY INFORMATION")
        print("="*60)
        display(seasons_display)
    else:
        print("No season members found in data")
else:
    print("Season data not available")

SEASONS

Total seasons: 3
SEASONS - KEY INFORMATION


id,year,number,defaultRegistrationStart,defaultRegistrationEnd,ageGroups
i64,i64,i64,str,str,list[str]
1,2024,61,null,null,"[""/api/seasonAgeGroups/1"", ""/api/seasonAgeGroups/2"", … ""/api/seasonAgeGroups/8""]"
2,2025,62,null,null,"[""/api/seasonAgeGroups/9"", ""/api/seasonAgeGroups/10"", … ""/api/seasonAgeGroups/16""]"
3,2026,63,"""2025-10-01T00:00:00+00:00""","""2025-11-15T00:00:00+00:00""","[""/api/seasonAgeGroups/17"", ""/api/seasonAgeGroups/18"", … ""/api/seasonAgeGroups/24""]"


In [8]:
seasons_df

@id,@type,id,year,number,previousSeason,successorSeason,readyToUse,ageGroups,defaultRegistrationStart,defaultRegistrationEnd,daysForRegistrationChangeDeadline
str,str,i64,i64,i64,struct[11],struct[10],bool,list[str],str,str,i64
"""/api/seasons/1""","""Season""",1,2024,61,null,"{""/api/seasons/2"",""Season"",2,2025,62,true,[""/api/seasonAgeGroups/9"", ""/api/seasonAgeGroups/10"", … ""/api/seasonAgeGroups/16""],null,null,null}",true,"[""/api/seasonAgeGroups/1"", ""/api/seasonAgeGroups/2"", … ""/api/seasonAgeGroups/8""]",null,null,null
"""/api/seasons/2""","""Season""",2,2025,62,"{""/api/seasons/1"",""Season"",1,2024,61,""/api/seasons/2"",true,[""/api/seasonAgeGroups/1"", ""/api/seasonAgeGroups/2"", … ""/api/seasonAgeGroups/8""],null,null,null}","{""/api/seasons/3"",""Season"",3,2026,63,true,[""/api/seasonAgeGroups/17"", ""/api/seasonAgeGroups/18"", … ""/api/seasonAgeGroups/24""],""2025-10-01T00:00:00+00:00"",""2025-11-15T00:00:00+00:00"",28}",true,"[""/api/seasonAgeGroups/9"", ""/api/seasonAgeGroups/10"", … ""/api/seasonAgeGroups/16""]",null,null,null
"""/api/seasons/3""","""Season""",3,2026,63,"{""/api/seasons/2"",""Season"",2,2025,62,""/api/seasons/3"",true,[""/api/seasonAgeGroups/9"", ""/api/seasonAgeGroups/10"", … ""/api/seasonAgeGroups/16""],null,null,null}",null,true,"[""/api/seasonAgeGroups/17"", ""/api/seasonAgeGroups/18"", … ""/api/seasonAgeGroups/24""]","""2025-10-01T00:00:00+00:00""","""2025-11-15T00:00:00+00:00""",28


## Work on timetable

In [9]:
# Extract and analyze timetable data
if 'raw_data' in dir() and 'timetable' in raw_data:
    timetable_data = raw_data['timetable']
    
    # Extract the timetable members
    if 'hydra:member' in timetable_data:
        timetable_members = timetable_data['hydra:member']
        
        print("="*60)
        print("TIMETABLE ANALYSIS")
        print("="*60)
        print(f"\nTotal timetable entries: {timetable_data.get('hydra:totalItems', len(timetable_members))}")
        
        # Examine first few entries to understand structure
        if timetable_members:
            print(f"\nFirst entry keys: {list(timetable_members[0].keys())}")
            print(f"\nSample entry:")
            for key, value in list(timetable_members[0].items())[:5]:
                print(f"  {key}: {value}")
        
        # Convert to DataFrame for analysis
        timetable_df = pl.DataFrame(timetable_members)
        
        print(f"\nTimetable DataFrame shape: {timetable_df.shape}")
        print(f"Columns: {timetable_df.columns}")
        
        # Show key relationships
        print("\n" + "="*60)
        print("KEY RELATIONSHIPS")
        print("="*60)
        
        # Analyze relationships with seasons
        if 'season' in timetable_df.columns:
            season_counts = timetable_df.group_by('season').agg(pl.len().alias('count')).sort('count', descending=True)
            print("\nEntries per season:")
            print(season_counts)
        
        # Display sample of important columns
        important_cols = [col for col in timetable_df.columns if col not in ['@context', '@id', '@type']][:8]
        print("\nSample timetable entries:")
        print(timetable_df.select(important_cols).head())
    else:
        print("No timetable members found in data")
else:
    print("Timetable data not available")

TIMETABLE ANALYSIS

Total timetable entries: 14289

First entry keys: ['@id', '@type', 'id', 'date', 'room', 'seasonAgeGroup', 'category', 'appointments', 'contest']

Sample entry:
  @id: /api/timetable_groups/5197
  @type: TimetableGroup
  id: 5197
  date: 2024-01-09
  room: {'@id': '/api/rooms/1578', '@type': 'Room', 'venue': {'@id': '/api/venues/381', '@type': 'Venue', 'id': 381, 'name': 'Deutsche Schule London'}, 'name': 'Forum'}



thread '<unnamed>' (4541476) panicked at crates/polars-core/src/frame/row/av_buffer.rs:279:30:
called `Result::unwrap()` on an `Err` value: ComputeError(ErrString("dtypes don't match, got struct[9], expected: struct[9]"))
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


PanicException: called `Result::unwrap()` on an `Err` value: ComputeError(ErrString("dtypes don't match, got struct[9], expected: struct[9]"))

In [10]:
timetable_data = raw_data['timetable']
timetable_members = timetable_data['hydra:member']

In [11]:
# Extract first timetable entry
first_entry = timetable_members[0]

print("="*60)
print("FIRST TIMETABLE ENTRY")
print("="*60)
print(f"\nEntry ID: {first_entry.get('@id')}")
print(f"Date: {first_entry.get('date')}")

room = first_entry.get('room', {})
print(f"Room: {room.get('name')} at {room.get('venue', {}).get('name')} (ID: {room.get('@id')})")
print(f"{first_entry.get('category', {}).get('name')} - Altersgruppe: {first_entry.get('seasonAgeGroup', {}).get('ageGroup')}")

FIRST TIMETABLE ENTRY

Entry ID: /api/timetable_groups/5197
Date: 2024-01-09
Room: Forum at Deutsche Schule London (ID: /api/rooms/1578)
Duo: Klavier und ein Streichinstrument - Altersgruppe: III


In [12]:
first_appointment = first_entry.get('appointments', [])[0]
appointment_entry = first_appointment.get('entry', {})
print(f"{first_appointment.get('startTime')} - {first_appointment.get('endTime')}")

print(f"Program title: {appointment_entry.get('programTitle')}")
for program_items in appointment_entry.get('programItems', []):
    print(f"{program_items.get('@id')} {program_items.get('title')}, {program_items.get('opus')}")


2024-01-09T09:40:00+00:00 - 2024-01-09T10:00:00+00:00
Program title: None
/api/programItems/134059 Sonata, op. 38/1
/api/programItems/134060 Marche Funèbre, op. 97/2


In [13]:
# Convert programItems into a detailed table with contributors and appearances
program_items_list = []

# Format appearances
appearances_str = '\n'.join([
    f"{a.get('person', {}).get('firstName', '')} {a.get('person', {}).get('lastName', '')} ({a.get('person', {}).get('city', '')}) - {a.get('instruments', [])[0].get('name', '')}"
    for a in appointment_entry.get('appearances', [])
])

for program_item in appointment_entry.get('programItems', []):
    # Format contributors
    contributors_str = '\n'.join([
        f"{c.get('name', '')} ({c.get('startYear', '')}-{c.get('endYear', '')})"
        for c in program_item.get('contributors', [])
    ])
    
    program_items_list.append({
        '@id': program_item.get('@id'),
        'title': program_item.get('title'),
        'opus': program_item.get('opus'),
        'duration': program_item.get('duration'),
        'epoch': program_item.get('epoch'),
        'tempo': program_item.get('tempo'),
        'style': program_item.get('style'),
        'key': program_item.get('key'),
        'regionOfOrigin': program_item.get('regionOfOrigin'),
        'technique': program_item.get('technique'),
        'catalogue': program_item.get('catalogue'),
        'contributors': contributors_str,
        'appearances': appearances_str
    })

program_items_df = pl.DataFrame(program_items_list)
display(program_items_df)# Extract and transform programItems into a detailed table

@id,title,opus,duration,epoch,tempo,style,key,regionOfOrigin,technique,catalogue,contributors,appearances
str,str,str,i64,str,str,null,str,null,null,null,str,str
"""/api/programItems/134059""","""Sonata""","""op. 38/1""",900,null,null,null,"""e-Moll""",null,null,null,"""Bernhard Romberg (1767-1841)""","""Dora Ollivier Alarcon (Richmond) - Klavier Julius Kaesbach (Kingston) - Violoncello"""
"""/api/programItems/134060""","""Marche Funèbre""","""op. 97/2""",300,"""d""","""Lento ma non troppo""",null,"""e-Moll""",null,null,null,"""Georg Goltermann (1824-1898)""","""Dora Ollivier Alarcon (Richmond) - Klavier Julius Kaesbach (Kingston) - Violoncello"""


In [14]:
test_member = timetable_members[0]
print(test_member)

{'@id': '/api/timetable_groups/5197', '@type': 'TimetableGroup', 'id': 5197, 'date': '2024-01-09', 'room': {'@id': '/api/rooms/1578', '@type': 'Room', 'venue': {'@id': '/api/venues/381', '@type': 'Venue', 'id': 381, 'name': 'Deutsche Schule London'}, 'name': 'Forum'}, 'seasonAgeGroup': {'@id': '/api/seasonAgeGroups/4', '@type': 'SeasonAgeGroup', 'id': 4, 'ageGroup': 'III'}, 'category': {'@id': '/api/categories/16', '@type': 'Category', 'id': 16, 'name': 'Duo: Klavier und ein Streichinstrument'}, 'appointments': [{'@id': '/api/appointments/27109', '@type': 'Appointment', 'id': 27109, 'startTime': '2024-01-09T09:40:00+00:00', 'startDate': '2024-01-09', 'endTime': '2024-01-09T10:00:00+00:00', 'endDate': '2024-01-09', 'entry': {'@id': '/api/entries/15384', '@type': 'Entry', 'id': 15384, 'programTitle': None, 'programItems': [{'@id': '/api/programItems/134059', '@type': 'ProgramItem', 'id': 134059, 'title': 'Sonata', 'opus': 'op. 38/1', 'duration': 900, 'epoch': None, 'openingLine': None, '

In [15]:
# Transform timetable data into normalized structure with foreign keys

# 1. Extract unique rooms
rooms_list = []
rooms_seen = set()

for member in timetable_members:
    room = member.get('room', {})
    room_id = room.get('@id')
    if room_id and room_id not in rooms_seen:
        rooms_seen.add(room_id)
        venue = room.get('venue', {})
        rooms_list.append({
            'room_id': room_id,
            'room_name': room.get('name'),
            'venue_id': venue.get('@id'),
            'venue_name': venue.get('name')
        })

rooms_df = pl.DataFrame(rooms_list)

# 2. Extract unique categories
categories_list = []
categories_seen = set()

for member in timetable_members:
    category = member.get('category', {})
    cat_id = category.get('@id')
    if cat_id and cat_id not in categories_seen:
        categories_seen.add(cat_id)
        categories_list.append({
            'category_id': cat_id,
            'category_name': category.get('name')
        })

categories_df = pl.DataFrame(categories_list)

# 3. Extract unique season age groups
season_age_groups_list = []
sag_seen = set()

for member in timetable_members:
    sag = member.get('seasonAgeGroup', {})
    try:
        sag_id = sag.get('@id')
    except Exception:
        sag_id = None
    if sag_id and sag_id not in sag_seen:
        sag_seen.add(sag_id)
        season_age_groups_list.append({
            'season_age_group_id': sag_id,
            'age_group': sag.get('ageGroup')
        })

season_age_groups_df = pl.DataFrame(season_age_groups_list)

# 4. Extract unique contests
contests_list = []
contests_seen = set()

for member in timetable_members:
    contest = member.get('contest', {})
    contest_id = contest.get('@id')
    if contest_id and contest_id not in contests_seen:
        contests_seen.add(contest_id)
        contests_list.append({
            'contest_id': contest_id,
            'contest_identifier': contest.get('identifier'),
            'time_zone': contest.get('timeZone')
        })

contests_df = pl.DataFrame(contests_list)

# 5. Main timetable groups table
timetable_groups_list = []

for member in timetable_members:
    sag_id = None
    try:
        sag_id = member.get('seasonAgeGroup', {}).get('@id')
    except Exception:
        pass

    timetable_groups_list.append({
        'timetable_group_id': member.get('@id'),
        'group_id': member.get('id'),
        'date': member.get('date'),
        'room_id': member.get('room', {}).get('@id'),
        'season_age_group_id': sag_id,
        'category_id': member.get('category', {}).get('@id'),
        'contest_id': member.get('contest', {}).get('@id'),
        'appointment_count': len(member.get('appointments', []))
    })

timetable_groups_df = pl.DataFrame(timetable_groups_list)

# Display normalized tables
print("="*60)
print("NORMALIZED TIMETABLE STRUCTURE")
print("="*60)

print("\n1. ROOMS:")
print(f"Shape: {rooms_df.shape}")
print(rooms_df)

print("\n2. CATEGORIES:")
print(f"Shape: {categories_df.shape}")
print(categories_df)

print("\n3. SEASON AGE GROUPS:")
print(f"Shape: {season_age_groups_df.shape}")
print(season_age_groups_df)

print("\n4. CONTESTS:")
print(f"Shape: {contests_df.shape}")
print(contests_df)

print("\n5. TIMETABLE GROUPS (with foreign keys):")
print(f"Shape: {timetable_groups_df.shape}")
display(timetable_groups_df.head(10))

NORMALIZED TIMETABLE STRUCTURE

1. ROOMS:
Shape: (1713, 4)
shape: (1_713, 4)
┌─────────────────┬─────────────────┬──────────────────┬───────────────────────────────────────┐
│ room_id         ┆ room_name       ┆ venue_id         ┆ venue_name                            │
│ ---             ┆ ---             ┆ ---              ┆ ---                                   │
│ str             ┆ str             ┆ str              ┆ str                                   │
╞═════════════════╪═════════════════╪══════════════════╪═══════════════════════════════════════╡
│ /api/rooms/1578 ┆ Forum           ┆ /api/venues/381  ┆ Deutsche Schule London                │
│ /api/rooms/1461 ┆ Aula            ┆ /api/venues/367  ┆ Deutsche Schule Helsinki              │
│ /api/rooms/1765 ┆ Orgel           ┆ /api/venues/382  ┆ St Peter Church                       │
│ /api/rooms/612  ┆ Kurt-Masur-Saal ┆ /api/venues/69   ┆ Musikschule Leipzig "Johann Sebastian │
│                 ┆                 ┆             

timetable_group_id,group_id,date,room_id,season_age_group_id,category_id,contest_id,appointment_count
str,i64,str,str,str,str,str,i64
"""/api/timetable_groups/5197""",5197,"""2024-01-09""","""/api/rooms/1578""","""/api/seasonAgeGroups/4""","""/api/categories/16""","""/api/contests/185""",1
"""/api/timetable_groups/5196""",5196,"""2024-01-09""","""/api/rooms/1578""","""/api/seasonAgeGroups/5""","""/api/categories/9""","""/api/contests/185""",1
"""/api/timetable_groups/3430""",3430,"""2024-01-09""","""/api/rooms/1461""","""/api/seasonAgeGroups/1""","""/api/categories/47""","""/api/contests/183""",6
"""/api/timetable_groups/3429""",3429,"""2024-01-09""","""/api/rooms/1461""","""/api/seasonAgeGroups/2""","""/api/categories/47""","""/api/contests/183""",21
"""/api/timetable_groups/3572""",3572,"""2024-01-09""","""/api/rooms/1461""","""/api/seasonAgeGroups/3""","""/api/categories/47""","""/api/contests/183""",7
"""/api/timetable_groups/5198""",5198,"""2024-01-09""","""/api/rooms/1578""","""/api/seasonAgeGroups/5""","""/api/categories/4""","""/api/contests/185""",1
"""/api/timetable_groups/5199""",5199,"""2024-01-09""","""/api/rooms/1578""","""/api/seasonAgeGroups/6""","""/api/categories/4""","""/api/contests/185""",2
"""/api/timetable_groups/5200""",5200,"""2024-01-09""","""/api/rooms/1765""","""/api/seasonAgeGroups/6""","""/api/categories/12""","""/api/contests/185""",1
"""/api/timetable_groups/5651""",5651,"""2024-01-09""","""/api/rooms/1578""","""/api/seasonAgeGroups/3""","""/api/categories/8""","""/api/contests/185""",1


In [16]:
# Extract and normalize appointments, program items, contributors, and appearances

# 1. Extract appointments with their relationships
appointments_list = []
program_items_list = []
program_item_contributors_list = []
appearances_list = []
persons_list = []
instruments_list = []

persons_seen = set()
instruments_seen = set()

for timetable_group in timetable_members:
    group_id = timetable_group.get('@id')
    
    for appointment in timetable_group.get('appointments', []):
        appointment_id = appointment.get('@id')
        entry = appointment.get('entry', {})
        entry_id = entry.get('@id')
        
        # Add appointment
        appointments_list.append({
            'appointment_id': appointment_id,
            'timetable_group_id': group_id,
            'entry_id': entry_id,
            'start_time': appointment.get('startTime'),
            'end_time': appointment.get('endTime')
        })
        
        # Extract program items
        for program_item in entry.get('programItems', []):
            pi_id = program_item.get('@id')
            program_items_list.append({
                'program_item_id': pi_id,
                'entry_id': entry_id,
                'title': program_item.get('title'),
                'opus': program_item.get('opus'),
                'duration': program_item.get('duration'),
                'epoch': program_item.get('epoch'),
                'tempo': program_item.get('tempo'),
                'style': program_item.get('style'),
                'key': program_item.get('key'),
                'region_of_origin': program_item.get('regionOfOrigin'),
                'technique': program_item.get('technique'),
                'catalogue': program_item.get('catalogue')
            })
            
            # Extract contributors for program item
            # TODO: The contributors do not have a unique id in the original data. Better to keep it as a struct/json column in the program_items tables
            for contributor in program_item.get('contributors', []):
                program_item_contributors_list.append({
                    'program_item_id': pi_id,
                    'contributor_name': contributor.get('name'),
                    'start_year': contributor.get('startYear'),
                    'end_year': contributor.get('endYear')
                })
        
        # Extract appearances and persons
        for appearance in entry.get('appearances', []):
            appearance_id = appearance.get('@id')
            person = appearance.get('person', {})
            person_id = person.get('@id')
            
            # Add person if not seen
            if person_id and person_id not in persons_seen:
                persons_seen.add(person_id)
                persons_list.append({
                    'person_id': person_id,
                    'first_name': person.get('firstName'),
                    'last_name': person.get('lastName'),
                    'city': person.get('city'),
                    'country': person.get('country')
                })
            
            # Add appearance
            appearances_list.append({
                'appearance_id': appearance_id,
                'entry_id': entry_id,
                'person_id': person_id
            })
            
            # Extract instruments for appearance
            for instrument in appearance.get('instruments', []):
                instr_id = instrument.get('@id')
                if instr_id and instr_id not in instruments_seen:
                    instruments_seen.add(instr_id)
                    instruments_list.append({
                        'instrument_id': instr_id,
                        'name': instrument.get('name'),
                        'family': instrument.get('family')
                    })
                
                # Link appearance to instrument
                if instr_id:
                    appearances_list.append({
                        'appearance_id': appearance_id,
                        'entry_id': entry_id,
                        'person_id': person_id,
                        'instrument_id': instr_id
                    })

# Create DataFrames
program_items_schemas = {
    'program_item_id': pl.Utf8,
    'entry_id': pl.Utf8,
    'title': pl.Utf8,
    'opus': pl.Utf8,
    'duration': pl.Int16,
    'epoch': pl.Utf8,
    'tempo': pl.Utf8,
    'style': pl.Utf8,
    'key': pl.Utf8,
    'region_of_origin': pl.Utf8,
    'technique': pl.Utf8,
    'catalogue': pl.Utf8
}

appointments_df = pl.DataFrame(appointments_list) if appointments_list else pl.DataFrame()
program_items_df = pl.DataFrame(program_items_list, schema=program_items_schemas) if program_items_list else pl.DataFrame()
program_item_contributors_df = pl.DataFrame(program_item_contributors_list) if program_item_contributors_list else pl.DataFrame()
appearances_df = pl.DataFrame(appearances_list) if appearances_list else pl.DataFrame()
persons_df = pl.DataFrame(persons_list) if persons_list else pl.DataFrame()
instruments_df = pl.DataFrame(instruments_list) if instruments_list else pl.DataFrame()

# Display summary
print("="*60)
print("NORMALIZED APPOINTMENTS & RELATED DATA")
print("="*60)

print(f"\n1. APPOINTMENTS:")
print(f"Shape: {appointments_df.shape}")
print(appointments_df.head())

print(f"\n2. PROGRAM ITEMS:")
print(f"Shape: {program_items_df.shape}")
print(program_items_df.head())

print(f"\n3. PROGRAM ITEM CONTRIBUTORS:")
print(f"Shape: {program_item_contributors_df.shape}")
print(program_item_contributors_df.head())

print(f"\n4. PERSONS:")
print(f"Shape: {persons_df.shape}")
print(persons_df.head())

print(f"\n5. INSTRUMENTS:")
print(f"Shape: {instruments_df.shape}")
print(instruments_df)

print(f"\n6. APPEARANCES:")
print(f"Shape: {appearances_df.shape}")
print(appearances_df.head())

NORMALIZED APPOINTMENTS & RELATED DATA

1. APPOINTMENTS:
Shape: (38043, 5)
shape: (5, 5)
┌───────────────────┬───────────────────┬───────────────────┬───────────────────┬──────────────────┐
│ appointment_id    ┆ timetable_group_i ┆ entry_id          ┆ start_time        ┆ end_time         │
│ ---               ┆ d                 ┆ ---               ┆ ---               ┆ ---              │
│ str               ┆ ---               ┆ str               ┆ str               ┆ str              │
│                   ┆ str               ┆                   ┆                   ┆                  │
╞═══════════════════╪═══════════════════╪═══════════════════╪═══════════════════╪══════════════════╡
│ /api/appointments ┆ /api/timetable_gr ┆ /api/entries/1538 ┆ 2024-01-09T09:40: ┆ 2024-01-09T10:00 │
│ /27109            ┆ oups/5197         ┆ 4                 ┆ 00+00:00          ┆ :00+00:00        │
│ /api/appointments ┆ /api/timetable_gr ┆ /api/entries/1070 ┆ 2024-01-09T09:20: ┆ 2024-01-09T09:35 │
│ 

In [17]:
# Search persons_df for 'Straub' (case-insensitive)
if 'persons_df' in dir() and persons_df.shape[0] > 0:
    q = "straub"
    matches = persons_df.filter(
        pl.col('last_name').fill_null('').str.to_lowercase().str.contains('rehfeldt') #&
        # pl.col('first_name').fill_null('').str.to_lowercase().str.contains('julius') 
    )
    print(f"Found {matches.shape[0]} match(es)")
    display(matches)
else:
    print("persons_df not available or empty")

Found 2 match(es)


person_id,first_name,last_name,city,country
str,str,str,str,null
"""/api/users/44472/persons/65911""","""Ferdinand""","""Rehfeldt""","""Münster""",null
"""/api/users/13812/persons/20009""","""Julius""","""Rehfeldt""","""Plochingen""",null


In [18]:
display(persons_df)

person_id,first_name,last_name,city,country
str,str,str,str,null
"""/api/users/7749/persons/10831""","""Dora""","""Ollivier Alarcon""","""Richmond""",null
"""/api/users/17927/persons/25917""","""Julius""","""Kaesbach""","""Kingston""",null
"""/api/users/1547/persons/1974""","""Oscar""","""Domann""","""Kingston upon Thames""",null
"""/api/users/18845/persons/26986""","""Frida""","""Faugel""","""Helsinki""",null
"""/api/users/18854/persons/27001""","""Lucia""","""Fenech""","""Helsinki""",null
…,…,…,…,…
"""/api/users/46312/persons/68331""","""Hrache""","""Ghukasyan""","""Bremerhaven """,null
"""/api/users/46035/persons/67916""","""Kassandra""","""Suhl""","""Berlin""",null
"""/api/users/46278/persons/68282""","""Miara""","""Piotrowski""","""Strausberg""",null
